In [2]:
import tensorflow as tf

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

In [4]:
# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

In [5]:
# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

In [6]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

#### Convolution:

The tf.nn.conv2d() function computes the convolution against weight W as shown above.

In [7]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

the code is using tf.nn.bias_add() to add the bias. Using tf.add() doesn't work when the tensors aren't the same shape.

#### Max-Pooling:

In [9]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In the code below, we're creating 3 layers alternating between convolutions and max pooling followed by a fully connected and output layer.

In [10]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

#### Session:

In [14]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [15]:
# Model
logits = conv_net(x, weights, biases, keep_prob)

In [16]:
# Define loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [17]:
# Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [18]:
# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
# Initializing the variables
init = tf. global_variables_initializer()

In [20]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 85739.6953 Validation Accuracy: 0.117188
Epoch  1, Batch   2 -Loss: 65299.6641 Validation Accuracy: 0.105469
Epoch  1, Batch   3 -Loss: 57992.5547 Validation Accuracy: 0.082031
Epoch  1, Batch   4 -Loss: 46342.3516 Validation Accuracy: 0.097656
Epoch  1, Batch   5 -Loss: 41370.3633 Validation Accuracy: 0.085938
Epoch  1, Batch   6 -Loss: 40483.6133 Validation Accuracy: 0.093750
Epoch  1, Batch   7 -Loss: 35593.3477 Validation Accuracy: 0.089844
Epoch  1, Batch   8 -Loss: 31002.9141 Validation Accuracy: 0.093750
Epoch  1, Batch   9 -Loss: 29596.0664 Validation Accuracy: 0.097656
Epoch  1, Batch  10 -Loss: 28257.7617 Validation Accuracy: 0.097656
Epoch  1, Batch  11 -Loss: 25896.7129 Validation Accuracy: 0.113281
Epoch  1, Batch  12 -Loss: 23371.8926 Validation Accuracy: 0.113281
Epoch  1, Batch  13 -Loss: 21890.3555 Validation Accuracy: 0.132812
Epoch  1, Batch  14 -Loss: 21186.4180 Validation Accuracy: 0.136719
Epoch  1, Batch  15 -Loss: 20102.6113 Validation

Epoch  1, Batch 122 -Loss:  4443.5020 Validation Accuracy: 0.468750
Epoch  1, Batch 123 -Loss:  3375.4099 Validation Accuracy: 0.476562
Epoch  1, Batch 124 -Loss:  4212.3164 Validation Accuracy: 0.476562
Epoch  1, Batch 125 -Loss:  4406.7666 Validation Accuracy: 0.476562
Epoch  1, Batch 126 -Loss:  4261.5918 Validation Accuracy: 0.468750
Epoch  1, Batch 127 -Loss:  3866.6260 Validation Accuracy: 0.476562
Epoch  1, Batch 128 -Loss:  4441.1836 Validation Accuracy: 0.480469
Epoch  1, Batch 129 -Loss:  4044.3379 Validation Accuracy: 0.476562
Epoch  1, Batch 130 -Loss:  3457.8027 Validation Accuracy: 0.480469
Epoch  1, Batch 131 -Loss:  4154.6191 Validation Accuracy: 0.484375
Epoch  1, Batch 132 -Loss:  3866.6255 Validation Accuracy: 0.476562
Epoch  1, Batch 133 -Loss:  3733.9819 Validation Accuracy: 0.476562
Epoch  1, Batch 134 -Loss:  3732.7349 Validation Accuracy: 0.484375
Epoch  1, Batch 135 -Loss:  4040.8215 Validation Accuracy: 0.480469
Epoch  1, Batch 136 -Loss:  3448.4482 Validation

Epoch  1, Batch 243 -Loss:  2175.0417 Validation Accuracy: 0.562500
Epoch  1, Batch 244 -Loss:  2607.0259 Validation Accuracy: 0.554688
Epoch  1, Batch 245 -Loss:  2144.9893 Validation Accuracy: 0.558594
Epoch  1, Batch 246 -Loss:  2239.4360 Validation Accuracy: 0.558594
Epoch  1, Batch 247 -Loss:  2632.7532 Validation Accuracy: 0.566406
Epoch  1, Batch 248 -Loss:  2021.9652 Validation Accuracy: 0.570312
Epoch  1, Batch 249 -Loss:  1920.5725 Validation Accuracy: 0.570312
Epoch  1, Batch 250 -Loss:  1910.0593 Validation Accuracy: 0.574219
Epoch  1, Batch 251 -Loss:  1953.2872 Validation Accuracy: 0.574219
Epoch  1, Batch 252 -Loss:  2696.4316 Validation Accuracy: 0.578125
Epoch  1, Batch 253 -Loss:  1879.9353 Validation Accuracy: 0.578125
Epoch  1, Batch 254 -Loss:  2432.5005 Validation Accuracy: 0.570312
Epoch  1, Batch 255 -Loss:  1840.4113 Validation Accuracy: 0.566406
Epoch  1, Batch 256 -Loss:  3024.1260 Validation Accuracy: 0.566406
Epoch  1, Batch 257 -Loss:  2097.8911 Validation

Epoch  1, Batch 364 -Loss:  1337.9734 Validation Accuracy: 0.628906
Epoch  1, Batch 365 -Loss:  1831.7224 Validation Accuracy: 0.628906
Epoch  1, Batch 366 -Loss:  1684.8260 Validation Accuracy: 0.625000
Epoch  1, Batch 367 -Loss:  1523.9326 Validation Accuracy: 0.628906
Epoch  1, Batch 368 -Loss:  1401.0840 Validation Accuracy: 0.640625
Epoch  1, Batch 369 -Loss:  1450.1492 Validation Accuracy: 0.636719
Epoch  1, Batch 370 -Loss:  1808.9380 Validation Accuracy: 0.636719
Epoch  1, Batch 371 -Loss:  1508.2577 Validation Accuracy: 0.632812
Epoch  1, Batch 372 -Loss:  1099.1796 Validation Accuracy: 0.640625
Epoch  1, Batch 373 -Loss:  1631.0455 Validation Accuracy: 0.636719
Epoch  1, Batch 374 -Loss:  1676.4822 Validation Accuracy: 0.648438
Epoch  1, Batch 375 -Loss:  1975.4941 Validation Accuracy: 0.652344
Epoch  1, Batch 376 -Loss:  2070.9438 Validation Accuracy: 0.636719
Epoch  1, Batch 377 -Loss:  1839.9032 Validation Accuracy: 0.640625
Epoch  1, Batch 378 -Loss:  1073.1221 Validation

Epoch  2, Batch  56 -Loss:  1583.5056 Validation Accuracy: 0.652344
Epoch  2, Batch  57 -Loss:  1314.4142 Validation Accuracy: 0.644531
Epoch  2, Batch  58 -Loss:  1634.2939 Validation Accuracy: 0.644531
Epoch  2, Batch  59 -Loss:  1657.7869 Validation Accuracy: 0.648438
Epoch  2, Batch  60 -Loss:  1619.7283 Validation Accuracy: 0.652344
Epoch  2, Batch  61 -Loss:  1504.3020 Validation Accuracy: 0.644531
Epoch  2, Batch  62 -Loss:  1456.8458 Validation Accuracy: 0.648438
Epoch  2, Batch  63 -Loss:  1032.7825 Validation Accuracy: 0.656250
Epoch  2, Batch  64 -Loss:  1316.7935 Validation Accuracy: 0.648438
Epoch  2, Batch  65 -Loss:  1294.5579 Validation Accuracy: 0.656250
Epoch  2, Batch  66 -Loss:   897.0128 Validation Accuracy: 0.656250
Epoch  2, Batch  67 -Loss:  1327.8994 Validation Accuracy: 0.664062
Epoch  2, Batch  68 -Loss:  1131.6819 Validation Accuracy: 0.667969
Epoch  2, Batch  69 -Loss:  1381.2183 Validation Accuracy: 0.671875
Epoch  2, Batch  70 -Loss:  1277.7418 Validation

...

Epoch  9, Batch 320 -Loss:   147.5380 Validation Accuracy: 0.808594
Epoch  9, Batch 321 -Loss:   245.2863 Validation Accuracy: 0.812500
Epoch  9, Batch 322 -Loss:   278.8036 Validation Accuracy: 0.812500
Epoch  9, Batch 323 -Loss:   167.3951 Validation Accuracy: 0.824219
Epoch  9, Batch 324 -Loss:   186.2643 Validation Accuracy: 0.816406
Epoch  9, Batch 325 -Loss:   320.8086 Validation Accuracy: 0.820312
Epoch  9, Batch 326 -Loss:   286.5481 Validation Accuracy: 0.804688
Epoch  9, Batch 327 -Loss:   313.5409 Validation Accuracy: 0.804688
Epoch  9, Batch 328 -Loss:   176.8028 Validation Accuracy: 0.816406
Epoch  9, Batch 329 -Loss:   253.4801 Validation Accuracy: 0.812500
Epoch  9, Batch 330 -Loss:   224.8132 Validation Accuracy: 0.816406
Epoch  9, Batch 331 -Loss:   251.8619 Validation Accuracy: 0.816406
Epoch  9, Batch 332 -Loss:   239.1079 Validation Accuracy: 0.824219
Epoch  9, Batch 333 -Loss:   225.3039 Validation Accuracy: 0.828125
Epoch  9, Batch 334 -Loss:   163.2088 Validation

Epoch 10, Batch  12 -Loss:   181.1441 Validation Accuracy: 0.812500
Epoch 10, Batch  13 -Loss:   195.9914 Validation Accuracy: 0.808594
Epoch 10, Batch  14 -Loss:   348.3012 Validation Accuracy: 0.812500
Epoch 10, Batch  15 -Loss:   153.8745 Validation Accuracy: 0.812500
Epoch 10, Batch  16 -Loss:   291.6796 Validation Accuracy: 0.816406
Epoch 10, Batch  17 -Loss:   170.4427 Validation Accuracy: 0.816406
Epoch 10, Batch  18 -Loss:   265.1762 Validation Accuracy: 0.804688
Epoch 10, Batch  19 -Loss:   233.2658 Validation Accuracy: 0.816406
Epoch 10, Batch  20 -Loss:   175.1994 Validation Accuracy: 0.812500
Epoch 10, Batch  21 -Loss:   173.9476 Validation Accuracy: 0.820312
Epoch 10, Batch  22 -Loss:   282.5148 Validation Accuracy: 0.812500
Epoch 10, Batch  23 -Loss:   174.0313 Validation Accuracy: 0.808594
Epoch 10, Batch  24 -Loss:   182.5462 Validation Accuracy: 0.824219
Epoch 10, Batch  25 -Loss:   237.8607 Validation Accuracy: 0.816406
Epoch 10, Batch  26 -Loss:   209.6026 Validation

Epoch 10, Batch 133 -Loss:   164.1812 Validation Accuracy: 0.808594
Epoch 10, Batch 134 -Loss:   226.1963 Validation Accuracy: 0.796875
Epoch 10, Batch 135 -Loss:   221.2351 Validation Accuracy: 0.792969
Epoch 10, Batch 136 -Loss:   216.1862 Validation Accuracy: 0.804688
Epoch 10, Batch 137 -Loss:   268.5282 Validation Accuracy: 0.816406
Epoch 10, Batch 138 -Loss:   128.0637 Validation Accuracy: 0.816406
Epoch 10, Batch 139 -Loss:   279.7549 Validation Accuracy: 0.789062
Epoch 10, Batch 140 -Loss:   195.5369 Validation Accuracy: 0.796875
Epoch 10, Batch 141 -Loss:   190.5796 Validation Accuracy: 0.792969
Epoch 10, Batch 142 -Loss:   284.5610 Validation Accuracy: 0.800781
Epoch 10, Batch 143 -Loss:   193.1279 Validation Accuracy: 0.796875
Epoch 10, Batch 144 -Loss:   196.4031 Validation Accuracy: 0.800781
Epoch 10, Batch 145 -Loss:   165.9505 Validation Accuracy: 0.800781
Epoch 10, Batch 146 -Loss:   259.1480 Validation Accuracy: 0.804688
Epoch 10, Batch 147 -Loss:   250.6210 Validation

Epoch 10, Batch 254 -Loss:   253.4523 Validation Accuracy: 0.816406
Epoch 10, Batch 255 -Loss:   135.7468 Validation Accuracy: 0.820312
Epoch 10, Batch 256 -Loss:   242.9547 Validation Accuracy: 0.812500
Epoch 10, Batch 257 -Loss:   124.5997 Validation Accuracy: 0.820312
Epoch 10, Batch 258 -Loss:   169.4646 Validation Accuracy: 0.816406
Epoch 10, Batch 259 -Loss:   247.4684 Validation Accuracy: 0.820312
Epoch 10, Batch 260 -Loss:   216.2782 Validation Accuracy: 0.816406
Epoch 10, Batch 261 -Loss:   171.4060 Validation Accuracy: 0.820312
Epoch 10, Batch 262 -Loss:   146.7727 Validation Accuracy: 0.812500
Epoch 10, Batch 263 -Loss:   167.4452 Validation Accuracy: 0.824219
Epoch 10, Batch 264 -Loss:   263.4519 Validation Accuracy: 0.816406
Epoch 10, Batch 265 -Loss:   185.6163 Validation Accuracy: 0.820312
Epoch 10, Batch 266 -Loss:   262.9829 Validation Accuracy: 0.824219
Epoch 10, Batch 267 -Loss:   209.4788 Validation Accuracy: 0.824219
Epoch 10, Batch 268 -Loss:   186.3558 Validation

Epoch 10, Batch 375 -Loss:   205.5016 Validation Accuracy: 0.828125
Epoch 10, Batch 376 -Loss:   201.6622 Validation Accuracy: 0.828125
Epoch 10, Batch 377 -Loss:   189.1444 Validation Accuracy: 0.828125
Epoch 10, Batch 378 -Loss:   116.8219 Validation Accuracy: 0.835938
Epoch 10, Batch 379 -Loss:   177.2477 Validation Accuracy: 0.828125
Epoch 10, Batch 380 -Loss:   187.0936 Validation Accuracy: 0.824219
Epoch 10, Batch 381 -Loss:   146.2749 Validation Accuracy: 0.824219
Epoch 10, Batch 382 -Loss:   185.0067 Validation Accuracy: 0.828125
Epoch 10, Batch 383 -Loss:   104.5978 Validation Accuracy: 0.828125
Epoch 10, Batch 384 -Loss:   166.3645 Validation Accuracy: 0.824219
Epoch 10, Batch 385 -Loss:   210.2863 Validation Accuracy: 0.820312
Epoch 10, Batch 386 -Loss:   123.8477 Validation Accuracy: 0.828125
Epoch 10, Batch 387 -Loss:   121.7377 Validation Accuracy: 0.828125
Epoch 10, Batch 388 -Loss:   190.9211 Validation Accuracy: 0.820312
Epoch 10, Batch 389 -Loss:   190.0929 Validation